<a href="https://colab.research.google.com/github/andy311p/Udemy_deep_learning_pytorch/blob/master/FeedForwardNNGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

train_dataset = dsets.MNIST(root='./data', 
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)
test_dataset = dsets.MNIST(root='./data', 
                            train=False,
                            transform=transforms.ToTensor(),
                            download=True)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!





/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
batch_size = 100
n_iters = 3000
num_epochs = int(n_iters / (len(train_dataset)/batch_size))

#iterable object
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [ ]:
class FeedForwardNeuralNetModel(nn.Module):
  def __init__(self,input_dim,hidden_dim,output_dim):
    super(FeedForwardNeuralNetModel,self).__init__()
    #Linear function
    self.fc1 = nn.Linear(input_dim,hidden_dim)

    #non Linear function
    #self.sigmoid = nn.Sigmoid()
    #self.tanh = nn.Tanh()
    self.relu1 = nn.ReLU()

    #adding second non linear layer
    self.fc2 = nn.Linear(hidden_dim,hidden_dim)
    self.relu2 = nn.ReLU()

    #Linear function (read out)
    self.fc3 = nn.Linear(hidden_dim,output_dim)

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu1(out)
    out = self.fc2(out)
    out = self.relu2(out)
    out = self.fc3(out)
    return out  

In [ ]:
#image size
input_dim = 28*28
hidden_dim = 500
#num of classes(digits 0-9)
output_dim=10

model = FeedForwardNeuralNetModel(input_dim,hidden_dim,output_dim).cuda()
#computes the softmax automatically
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
iter = 0
for epoch in range(num_epochs):
  for i, (images,labels) in enumerate(train_loader):
    images = Variable(images.view(-1,28*28).cuda())
    labels = Variable(labels.cuda())

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    iter += 1
    #every 500 iteration calculate accuracy
    if iter % 500 == 0:
      correct = 0.0
      total = 0.0
      #calc accuracy on test data
      for images, labels in test_loader:
        images = Variable(images.view(-1,28*28).cuda())
        outputs = model(images)
        _, predictions = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predictions.cpu() == labels.cpu()).sum()

      accuracy = 100 * correct / total
      print('iteration: {}. loss: {} Accuracy: {}'.format(iter, loss.data,accuracy)) 


iteration: 500. loss: 0.31977593898773193 Accuracy: 91.4800033569336
iteration: 1000. loss: 0.18561242520809174 Accuracy: 94.16000366210938
iteration: 1500. loss: 0.17743991315364838 Accuracy: 94.91000366210938
iteration: 2000. loss: 0.14217378199100494 Accuracy: 96.36000061035156
iteration: 2500. loss: 0.0911073237657547 Accuracy: 96.51000213623047
iteration: 3000. loss: 0.06030760332942009 Accuracy: 97.08999633789062
